In [1]:
import numpy as np
import math

import scipy as sp
from scipy.spatial.transform import Rotation
from scipy.spatial import distance
from scipy.spatial import KDTree

import time

import os

## Raup's model
Calculate the following values based on raup's model.
- Coordinates of generating curve
- Coordinates of center of generating curve
- Frenet frame

In [2]:
def RaupModel(W_r, T_r, D_r, S_r, delta_r, gamma_r, r0, theta, phi, a = 1):
    """Raup's model
    
    Calculate the coordinates of the shell surface（x, y, z）.
    
    Args:
        W_r: the whorl expansion rate
        T_r: the rate of helicocone translation 
        D_r: the position of the generating curve in relation to the coiling axis
        S_r: the aspect ratio of the generating curve
        delta_r: angle between the generating curve and the coiling axis
        gamma_r: angle between the generating curve and the shell radius
        r0: initial radius
        theta: coiling angle
        phi: angle of rotation about aperture

    Returns:
        X, Y, Z: coordinates of generating curve
        px, py, pz: coordinates of center of generating curve
        xi1, xi2, xi3: Frenet frame
    
    """
    
    w = W_r**(theta/(2*np.pi))
    px = w * (2*D_r/(1 - D_r) + 1 )*np.cos(theta)
    py = w * (2*D_r/(1 - D_r) + 1 )*np.sin(theta)
    pz = w * (2*T_r*(D_r/(1 - D_r) + 1) )
    
    theta_grid, phi_grid = np.meshgrid(theta, phi)
    
    ww = W_r**(theta_grid/(2*np.pi))

    X =r0*ww*(
        (a*np.sin(theta_grid)*(-S_r*np.cos(phi_grid)*np.sin(gamma_r)+np.cos(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)))/S_r+(np.cos(theta_grid))*(1-2*D_r/(-1+D_r)+a*np.cos(gamma_r)*np.cos(phi_grid)+a*np.sin(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
    )
    
    Y =r0*ww*(
        np.cos(theta_grid)*(a*np.cos(phi_grid)*np.sin(gamma_r)
                       -a*np.cos(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
        +np.sin(theta_grid)*(1-2*D_r/(-1+D_r)+a*np.cos(gamma_r)*np.cos(phi_grid)
                        +a*np.sin(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
    )
    Z =r0*ww * (-2*T_r/(-1+D_r) + a*np.cos(delta_r)*np.sin(phi_grid)/S_r)
    
    r=r0*w
    
    ##
    ##Frenet frame
    ##
    logw = np.log(W_r)
    xi1x=(
        (1+D_r)*(np.cos(theta)*logw-2*np.pi*np.sin(theta))
    )/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+(logw)**2))
    xi1y=(
        (1+D_r)*(2*np.pi*np.cos(theta)+logw*np.sin(theta))
    )/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+logw**2))
    xi1z=np.tile((2*T_r*logw)/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+logw**2)),len(theta))
    
    xi2x=-(2*np.pi*np.cos(theta)+logw*np.sin(theta))/np.sqrt(4*np.pi**2+logw**2)
    xi2y=(np.cos(theta)*logw-2*np.pi*np.sin(theta))/np.sqrt(4*np.pi**2+logw**2)
    xi2z=np.zeros(len(theta))
    
    
    xi1=np.array([xi1x,xi1y,xi1z]).transpose()
    xi2=np.array([xi2x,xi2y,xi2z]).transpose()
    xi1 = np.apply_along_axis(normalize, 1, xi1)
    xi2 = np.apply_along_axis(normalize, 1, xi2)
    xi3 = np.cross(xi1, xi2)
    xi3 = np.apply_along_axis(normalize, 1, xi3)
    
    return (X, Y, Z, px, py, pz,r,xi1, xi2, xi3)

## Convert variable
- Convert raup's model parameters$W_{R}, T_{R}, D_{R}, \Delta, \Gamma $ to growing tube model parameters$E_{G}, C_{G}, T_{G}, \delta , \gamma $.
- Convert $\theta$ to $l$.

In [3]:
def cvt_Raup2GT(W_r, T_r, D_r, delta_r, gamma_r):
    
    E_g = ((1 - D_r)*np.log(W_r))/(np.sqrt(4*(np.pi**2)*(1+D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2)))
    C_g = ((2*np.pi*(1 - D_r**2))*np.sqrt(4*np.pi**2+(np.log(W_r)**2)))/(4*(np.pi**2)*(1+D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2))
    T_g = (4*np.pi*T_r*(1 - D_r)*np.log(W_r))/(4*(np.pi**2)*(1 + D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2))
    
    delta_g = np.arcsin(
        (2*T_r*np.cos(delta_r)*np.log(W_r)*(2*np.pi*np.cos(gamma_r) - np.log(W_r)*np.sin(gamma_r))
         -(1+D_r)*(4*np.pi**2+np.log(W_r)**2)*np.sin(delta_r))/np.sqrt((4*np.pi**2+np.log(W_r)**2)*(4*(1+D_r)**2*np.pi**2+((1+D_r)**2+4*T_r**2)*np.log(W_r)**2))
    )
    
    gamma_g_num = np.sqrt(2)*np.cos(delta_r)*(np.cos(delta_r)*np.log(W_r)
                                        +2*np.pi*np.sin(gamma_r))
    gamma_g_denom = np.sqrt(
        (4*np.cos(delta_r)**2 *
         (8*(1 + D_r)**2 * np.pi**4 + T_r**2 * np.log(W_r)**2 * (np.cos(2*gamma_r)*(-4*np.pi**2 + np.log(W_r)**2) + 4*np.pi*np.log(W_r)*np.sin(2*gamma_r)))
         +np.log(W_r)*(8*np.pi**2 * ((1+D_r)**2 + 3* T_r**2 + 3*T_r**2 + (1+D_r-T_r)*(1+D_r+T_r)*np.cos(2*delta_r))*np.log(W_r) + ((1+D_r)**2 + 6*T_r**2 + ((1+D_r)**2 - 2*T_r**2)*np.cos(2*delta_r))*np.log(W_r)**3 
                       + 4*(1+D_r)*T_r*(4*np.pi**2 + np.log(W_r)**2)*(2*np.pi*np.cos(gamma_r) - np.log(W_r)*np.sin(gamma_r))*np.sin(2*gamma_r)))
        /((4*(1+D_r)**2)*np.pi**2 + ((1+D_r)**2 + 4*T_r**2)*np.log(W_r)**2)
    )
    gamma_g = -np.arcsin(-gamma_g_num/gamma_g_denom)
    
    return E_g, C_g, T_g, delta_g, gamma_g

def cvt_theta2s(theta, W_r, T_r, D_r):
    s = -np.sqrt(4*(1 + D_r)**2 * np.pi**2 + ((1 + D_r)**2 + 4* T_r**2)*np.log(W_r)**2)*np.log(W_r**(theta/(2*np.pi)))/((-1+D_r)*np.log(W_r))
    return s

In [4]:
def cvt_theta2l(theta,W_r,T_r,D_r,r0):
    l=-np.sqrt(4*(1 + D_r)**2 * np.pi**2 + ((1 + D_r)**2 + 4* T_r**2)*np.log(W_r)**2)*r0*(-1+W_r**(theta/(2*np.pi)))/((-1+D_r)*np.log(W_r))
    return l

## Growing tube model
Calculate the following values based on the growing tube model
- Frenet frame($\xi_{1}i,\xi_{2}i,\xi_{3}i$) inclined at angles of $\delta$ and $\gamma$
- Coordinates of shell surface and soft body

In [5]:
def normalize(vec):
    norm = np.linalg.norm(vec)
    return vec/norm

def GTModel(E_g, C_g, T_g, delta_g, gamma_g, r0, l, dphi, x0, y0, z0, xi10, xi20, xi30, n, a=1):
    """Growing tube model
    Calculate the Frenet frame inclined at angles of delta_g and gamma_g
    
    Args:
        E_g: the growth rate at a certain stage s
        C_g: the curvature standardized by the radius r of the tube at growth stage s
        T_g: the tortuosity standardized by the radius r of the tube at growth stage s
        delta_g: angle around xi2
        gamma_g: angle around xi3
        r0: initial value of radius
        l: arc length represented by s
        dphi: width of angle phi
        x0, y0, z0: initial coordinates
        xi10, xi20, xi30: initial value of Frenet frame
    
    Returns:
        s: growth stage
        r: radius at growth stage s
        px, py, pz: coordinates of center of generating curve
        xi1, xi2, xi3: Frenet frame
        xi1_i, xi2_i, xi3_i: Frenet frame inclined at angles of delta_g and gamma_g
        
    """
    
    s=np.log(1+(E_g*l/r0))/E_g
    r =r0*np.exp(E_g*s)

    
    D = (C_g**2+T_g**2)**0.5
    ED3E2pD2 = E_g*D**3*(E_g**2 + D**2)
    expEs = np.exp(E_g*s)
    sinDs = np.sin(D*s)
    cosDs = np.cos(D*s)
    

    ##
    ## Growth Trajectory
    ##
    P = r0*D*(((D**2) * (T_g**2) + (E_g**2) * (T_g**2) + C_g**2 * E_g**2 * cosDs 
               + E_g*D*(C_g**2)*sinDs)*expEs - D**2 * (E_g**2 + T_g**2))/ED3E2pD2
    Q = r0*C_g*D*E_g*(-expEs*(C_g**2 + T_g**2)*cosDs 
                      + D*(D + expEs*E_g*sinDs))/ED3E2pD2
    R = r0*C_g*T_g*D*(((E_g**2) + (D**2) - (E_g**2) * cosDs - E_g*D*sinDs)*expEs - D**2)/ED3E2pD2

    X0 = np.array([x0, y0, z0])
    R0 = np.array([xi10, xi20, xi30]).transpose()

    px, py, pz = np.transpose(X0 + np.transpose(np.dot(R0, np.array([P, Q, R]))))

    ##
    ## Frenet frame
    ##
    xi1 = np.dot(
        R0,
        np.array([(T_g**2 + C_g**2*cosDs)/D**2, C_g*sinDs/D, C_g*T_g*(1 - cosDs)/D**2])
    ).transpose()
    xi2 = np.dot(
        R0,
        np.array([(-C_g*sinDs/D), cosDs, T_g*sinDs/D])
    ).transpose()
    xi3 = np.dot(
        R0,
        np.array([(C_g*T_g*(1 - cosDs))/D**2, (-T_g*sinDs)/D, (C_g**2+T_g**2*cosDs)/D**2])
    ).transpose()
    
    xi1 = np.apply_along_axis(normalize, 1, xi1)
    xi2 = np.apply_along_axis(normalize, 1, xi2)
    xi3 = np.apply_along_axis(normalize, 1, xi3)

    
    rot2 = Rotation.from_rotvec(delta_g*xi2).as_matrix()
    rot3 = Rotation.from_rotvec(gamma_g*xi3).as_matrix()
    rot_g = np.array([np.dot(rot3[i], rot2[i]) for i in range(len(rot2))])
    
    xi1_i  = np.array([normalize(np.dot(rot_g[i], xi1[i])) for i in range(len(xi1))])
    xi2_i  = np.array([normalize(np.dot(rot_g[i], xi2[i])) for i in range(len(xi2))])
    xi3_i  = np.array([normalize(np.dot(rot_g[i], xi3[i])) for i in range(len(xi3))])
    
    xi1_i = np.apply_along_axis(normalize, 1, xi1_i)
    xi2_i = np.apply_along_axis(normalize, 1, xi2_i)
    xi3_i = np.apply_along_axis(normalize, 1, xi3_i)

    return(s,r,px, py, pz, xi1, xi2, xi3,xi1_i,xi2_i,xi3_i)

## Rotate models

In [6]:
def translate2origin(coords_list,center_coords_aperture,xi1):
    """"
    Args:
    coords_list: 1D array of voxel center coordinates
    center_coords_aperture: coordinate of center of aperture_softbody
    
    Returns:
    cords_list2: 1D array of translated and rotated coords_list 
    
    """
    xx=(coords_list-center_coords_aperture)[:,0]
    yy=(coords_list-center_coords_aperture)[:,1]
    zz=(coords_list-center_coords_aperture)[:,2]
    
    X=[1,0,0]
    Y=[0,1,0]
    Z=[0,0,-1]
    

    if xi1[0]==0 and xi1[1]==0 and xi1[2]==1:
        R3=np.array([[1,0,0],[0,1,0],[0,0,-1]])
    elif xi1[0]==0 and xi1[1]==0 and xi1[2]==-1:
        R3=np.array([[1,0,0],[0,1,0],[0,0,1]])
        
    else:
        xi11=np.array([xi1[0],xi1[1],0])
        costh=np.dot(xi1,xi11)/(np.linalg.norm(xi11)*np.linalg.norm(xi1))
        Xx=np.cross(xi1,Z)/np.linalg.norm(np.cross(xi1,Z))
        
        if costh>1:
            costh=1
            
        if xi1[2]>0:
            th=0.5*np.pi+np.arccos(costh)

        elif xi1[2]<0:
            th=0.5*np.pi-np.arccos(costh)
        
        R3=Rotation.from_rotvec((th)*Xx).as_matrix()

    XX=R3[0,0]*xx+R3[0,1]*yy+R3[0,2]*zz
    YY=R3[1,0]*xx+R3[1,1]*yy+R3[1,2]*zz
    ZZ=R3[2,0]*xx+R3[2,1]*yy+R3[2,2]*zz
    
    coords_list2=np.array([XX,YY,ZZ]).T
    
    return(coords_list2)

## Calculate norm

In [7]:
def G_norm(coords_list,V):
    
    x=coords_list[:,0]
    y=coords_list[:,1]
    z=coords_list[:,2]
    
    ##
    ##Rotate about the z-axis to lpcate the center of mass(G) on the y-z plane
    ##
    G=np.sum(coords_list,axis=0)/len(coords_list)
    Gg=np.array([G[0], G[1], 0])
    
    th2=np.arccos(np.dot([0,-1,0],Gg)/np.linalg.norm(Gg))
    if Gg[0]>0:
        R4=Rotation.from_rotvec((2*np.pi-th2)*np.array([0,0,1])).as_matrix()
    elif Gg[0]<0:
        R4=Rotation.from_rotvec((th2)*np.array([0,0,1])).as_matrix()
    else:
        R4=np.array([[1,0,0],[0,1,0],[0,0,1]])

    XX=R4[0,0]*x+R4[0,1]*y+R4[0,2]*z
    YY=R4[1,0]*x+R4[1,1]*y+R4[1,2]*z
    ZZ=R4[2,0]*x+R4[2,1]*y+R4[2,2]*z

    ##
    ##Translate along the z-axis to place the point with the minimum z-value(qm) on the x-y plane.
    ##
    q_list=[]
    xi1e=np.array([0,0,-1])
    for i in range(0,len(XX)):
        A=np.array([XX[i],YY[i],ZZ[i]])
        q=np.dot(xi1e,(A))
        q_list.append(q)
    qm=np.array(q_list).max()
    GG=np.array([np.sum(XX)/len(XX), np.sum(YY)/len(YY), np.sum(ZZ)/len(ZZ)+qm])
    
    G_norm=np.linalg.norm(GG)/(V**(1/3))
    
    return(G_norm,XX,YY,ZZ+qm,GG,qm)

## Pramerter

In [23]:
num_w=2
_row=20
_col=20

Wr =10**(num_w*0.1)
Dr =0

gamr=0

deltarrange=np.linspace(-np.pi/2,np.pi/2,_row)
Trange=np.linspace(np.log10(0.1),np.log10(10),_col)

Tr,deltar=np.meshgrid(Trange,deltarrange)
ret=np.c_[10**(Tr.ravel()),deltar.ravel()]

Sr = 1
r0 = 1

theta_end=8*np.pi
theta = np.linspace(0, theta_end, 400)
Phi = np.linspace(0, 2*np.pi,100)
l_step0=800 

dphi=0.02

## Implementation

In [13]:
def functionalities(Wr,Tr,Dr,Sr,deltar,gamr,r0,theta,Phi,l_step0,dphi,Posture,filename):
    
    X, Y, Z, px, py, pz,r_r, xi1, xi2, xi3 = RaupModel(Wr, Tr, Dr, Sr, deltar,gamr, r0, theta, Phi)
    
    l_end=cvt_theta2l(theta[-1],Wr, Tr, Dr,r0)
    l_step=l_step0
    l=np.linspace(0,l_end,l_step)
    
    E_g, C_g, T_g, delta_g, gamma_g  = cvt_Raup2GT(Wr, Tr, Dr, deltar, gamr)
    s,r,px_g, py_g, pz_g, xi1_g, xi2_g, xi3_g,xi1_i,xi2_i,xi3_i=GTModel(E_g, C_g, T_g, delta_g, gamma_g, r0, l, dphi, px[0], py[0], pz[0], xi1[0], xi2[0], xi3[0],n=1,a=1)
    
    if deltar<0:
        if np.arccos(np.dot(xi1_g[0],[0,0,1]))<abs(deltar):
            TF=False
        else:    
            TF=True
    else:
        TF=True
    
    if TF==False:
        norm=(np.nan)
        projected_area=(np.nan)
        softtissue_rate=(np.nan)
        
    else:
        filename=f'{filename}.npz'
        modeldata=np.load(filename)
        dx,vox_coords,coords_list,vox_state,softtissue_rate=modeldata['dx'],modeldata['vox_coords'],modeldata['coords_list'],modeldata['vox_state'],modeldata['softtissue_rate']

        ##
        ##Calculate norm
        ##           
        center_coords_aperture=np.sum(coords_list[vox_state.ravel()==11],axis=0)/len(coords_list[vox_state.ravel()==11])
        if Posture=="Pgrowth":
            posture_vector=xi1_g[-1]
        elif Posture=="Paperture":
            posture_vector=xi1_i[-1]
        else:
            del(posture_vector)
            
        coords_list2=translate2origin(coords_list[vox_state.ravel()!=0],center_coords_aperture,posture_vector)
        shell_volume=len(coords_list2)*dx**(3)
        norm,XX,YY,ZZ,GG,qm=G_norm(coords_list2,shell_volume)
        
        ##
        ##Calculate projected area
        ##        
        target=np.array([XX,ZZ]).T
        
        x_max=XX.max()
        x_min=XX.min()
        z_max=ZZ.max()
        z_min=ZZ.min()

        Xa,Za=np.mgrid[x_min-dx:x_max+dx:dx,z_min-dx:z_max+dx:dx]
        tree_coords = np.array((Xa.ravel(), Za.ravel())).T
        tree=KDTree(tree_coords)

        kd_list=[]
        for m in range(0,len(target)):
            dist,index = tree.query(target[m])
            kd_list.append(index)
        
        number_area=len(np.unique(kd_list,axis=0))
        projected_area=(number_area*dx*dx)/(shell_volume**(2/3))
        
    return (norm,projected_area,softtissue_rate)

In [ ]:
norm_list_g=[]
area_list_g=[]

norm_list_i=[]
area_list_i=[]
softtissue_rate_list=[]

for i in range(0,len(ret)):
    filename=str(i)
    norm,projected_area,softtissue_rate=functionalities(Wr,ret[i,0],Dr,Sr,ret[i,1],gamr,r0,theta,Phi,l_step0,dphi,"Pgrowth",filename)
    norm_list_g.append(norm)
    area_list_g.append(projected_area)

    norm,projected_area,softtissue_rate=functionalities(Wr,ret[i,0],Dr,Sr,ret[i,1],gamr,r0,theta,Phi,l_step0,dphi,"Paperture",filename)
    norm_list_i.append(norm)
    area_list_i.append(projected_area)
    softtissue_rate_list.append(softtissue_rate)

In [27]:
np.savez("pgdata_xi1g",norm_list=norm_list_g,area_list=area_list_g)
np.savez("pgdata_xi1i",norm_list=norm_list_i,area_list=area_list_i)
np.save("softtissue_rate",softtissue_rate_list)